# BET Disk

This notebook walks through a **BET Disk** setup for the XV‑15 using Flow360. It covers loading geometry, defining mesh refinements including axisymmetric refinement for the rotor region, creating a BET disk from an external xrotor file and additional input parameters, as well as submitting the case through the Project interface.

The geometry consists of a BET disk in front of a wing.

## 1. Create Project from geometry
- Load Python libraries and Flow360 client. If you use environment variables or tokens, initialize them here so later API calls can authenticate. BET disk files are loaded here.
- Project is created from geometry CAD file

In [3]:
import flow360 as fl
from flow360.examples import TutorialBETDisk

TutorialBETDisk.get_files()

project = fl.Project.from_geometry(TutorialBETDisk.geometry, name="Tutorial BETDisk from Python")
geometry = project.geometry

# show face and edge groupings
geometry.show_available_groupings(verbose_mode=True)
geometry.group_faces_by_tag("faceName")
geometry.group_edges_by_tag("edgeName")

Output()

[16:17:21] INFO: Geometry successfully submitted:                                                                  
                   type   = Geometry                                                                               
                   name   = Tutorial BETDisk from Python                                                           
                   id     = geo-79a016dd-d687-4119-b39a-4a0916a8679e                                               
                   status = uploaded                                                                               
           

           INFO: Waiting for geometry to be processed.

Output()

[16:18:06] INFO:  >> Available attribute tags for grouping **faces**:

...

           INFO:            IDs: ['body00001_edge00004', 'body00001_edge00007', 'body00001_edge00011']

           INFO:  >> Available attribute tags for grouping **bodies**:

           INFO:     >> Tag '1': groupByFile. Grouping with this tag results in:

           INFO:         >> [0]: BET_tutorial_wing.csm

           INFO:            IDs: ['body00001']

## 2. Define BET disk refinements
We want to apply an axisymmetric refinement around the BET disk. To do so, we first define a cylinder volume zone and then we define the axisymmetric refinement in that cylinder. It is recommended to use a larger size for the BET disk axisymmetric refinement than the actual rotor, see the documentation theory for further reading.

In [4]:
with fl.SI_unit_system:
    cylinder1 = fl.Cylinder(
            name="cylinder1",
            axis=[1, 0, 0],
            center=[-2.0, 5.0, 0],
            outer_radius=4.0,
            height=0.6,
        )

    bet_disk_refinement = fl.AxisymmetricRefinement(
            name="BET_Disk",
            spacing_axial=0.02,
            spacing_radial=0.03,
            spacing_circumferential=0.06,
            entities=cylinder1,
        )

           INFO: using: SI unit system for unit inference.

## 3. Define meshing parameters
The global meshing parameters are defined with an automatic farfield and additional uniform refinement cylinders. We use the BET disk refinement inside the refinement list.

In [5]:
with fl.SI_unit_system:
    cylinder2 = fl.Cylinder(
        name="cylinder2",
        axis=[1, 0, 0],
        center=[0, 5, 0],
        outer_radius=4.1,
        height=5,
    )
    farfield = fl.AutomatedFarfield()
    meshing_params=fl.MeshingParams(
        defaults=fl.MeshingDefaults(
            surface_edge_growth_rate=1.2,
            surface_max_edge_length=0.5,
            curvature_resolution_angle=30 * fl.u.deg,
            boundary_layer_growth_rate=1.15,
            boundary_layer_first_layer_thickness=1e-06,
        ),
        volume_zones=[farfield],
        refinements=[
            bet_disk_refinement,
            fl.UniformRefinement(name="cylinder_refinement", spacing=0.1, entities=[cylinder2]),
            fl.SurfaceRefinement(
                name="tip",
                max_edge_length=0.01,
                faces=[
                    geometry["tip"],
                ],
            ),
            fl.SurfaceEdgeRefinement(
                name="aniso",
                method=fl.HeightBasedRefinement(value=0.0003),
                edges=[
                    geometry["wingTrailingEdge"],
                    geometry["wingLeadingEdge"],
                ],
            ),
            fl.SurfaceEdgeRefinement(
                name="projectAnisoSpacing",
                method=fl.ProjectAnisoSpacing(),
                edges=[
                    geometry["rootAirfoilEdge"],
                    geometry["tipAirfoilEdge"],
                ],
            ),
        ],
    )

           INFO: using: SI unit system for unit inference.

## 3. Define BET disk model
The BET disk model is defined from an xrotor file and is applied to the volume zone named 'bet_cylinder' consisiting of an approximation of the rotor disk. In this volume zone, the model will be applied.

In [6]:
with fl.SI_unit_system:
    bet_cylinder = fl.Cylinder(
        name="bet_cylinder",
        axis=[-1, 0, 0],
        center=[-2.0, 5.0, 0.0],
        outer_radius=3.81,
        height=0.4572,
    )

    bet_disk_model = fl.BETDisk.from_xrotor(
        file=fl.XROTORFile(file_path=TutorialBETDisk.extra["xrotor"]),
        rotation_direction_rule="rightHand",
        omega=460 * fl.u.rpm,
        chord_ref=0.3556,
        n_loading_nodes=20,
        entities=bet_cylinder,
        angle_unit=fl.u.deg,
        length_unit=fl.u.m,
    )

           INFO: using: SI unit system for unit inference.

## 4. Define SimulationParams
- The simulation parameters are defined in the python class fl.SimulationParams()
- BET disk is run if steady state time settings are used.

In [7]:
with fl.SI_unit_system:
    params = fl.SimulationParams(
        meshing=meshing_params,
        reference_geometry=fl.ReferenceGeometry(
            moment_center=[0.375, 0, 0],
            moment_length=[1.26666666, 1.26666666, 1.26666666],
            area=12.5,
        ),
        operating_condition=fl.AerospaceCondition.from_mach(
            mach=0.182,
            alpha=5 * fl.u.deg,
            reference_mach=0.54,
        ),
        time_stepping=fl.Steady(
            max_steps=10000, CFL=fl.RampCFL(initial=1, final=100, ramp_steps=2000)
        ),
        models=[
            fl.Wall(
                surfaces=[
                    geometry["wing"],
                    geometry["tip"],
                ],
            ),
            fl.Freestream(surfaces=farfield.farfield),
            fl.SlipWall(surfaces=farfield.symmetry_planes),
            fl.Fluid(
                navier_stokes_solver=fl.NavierStokesSolver(
                    absolute_tolerance=1e-12,
                ),
                turbulence_model_solver=fl.SpalartAllmaras(
                    absolute_tolerance=1e-10,
                    update_jacobian_frequency=1,
                    equation_evaluation_frequency=1,
                ),
            ),
            bet_disk_model,
        ],
        outputs=[
            fl.VolumeOutput(
                output_fields=[
                    "primitiveVars",
                    "betMetrics",
                    "qcriterion",
                ],
            ),
            fl.SurfaceOutput(
                surfaces=geometry["*"],
                output_fields=[
                    "primitiveVars",
                    "Cp",
                    "Cf",
                    "CfVec",
                ],
            ),
        ],
    )

           INFO: using: SI unit system for unit inference.

## 5. Run Case

In [8]:
project.run_case(params=params, name="Case of tutorial BETDisk from Python")

           INFO: using: SI unit system for unit inference.

[16:18:09] INFO: Successfully submitted:                                                                           
                   type   = Case                                                                                   
                   name   = Case of tutorial BETDisk from Python                                                   
                   id     = case-afbe238b-e4dc-416f-b305-1bb7afe5bec8                                              
                   status = pending                                                                                
           

CaseMeta(name='Case of tutorial BETDisk from Python', user_id='user-4141c6e3-e862-46b3-bc99-4848739f70b6', id='case-afbe238b-e4dc-416f-b305-1bb7afe5bec8', parent_id=None, solver_version='release-25.6', status=<Flow360Status.PENDING: 'pending'>, tags=[], created_at=None, updated_at=datetime.datetime(2025, 9, 10, 20, 18, 9, 127000, tzinfo=datetime.timezone.utc), updated_by=None, deleted=False, cloud_path_prefix=None, case_mesh_id='vm-b2bf0637-7562-4ee9-8af6-da37a7aa0f35', storageStatus='STANDARD', refId='vm-b2bf0637-7562-4ee9-8af6-da37a7aa0f35', elapsedTimeInSeconds=None, caseStartTime=None, caseTags=None, retryCount=None, caseParentId=None, combinedStatus='pending', parentFolderId='ROOT.FLOW360', caseSubmitTime='2025-09-10T20:18:08.895Z', parentFolders=[{'userId': 'user-4141c6e3-e862-46b3-bc99-4848739f70b6', 'name': 'ROOT.FLOW360', 'tags': None, 'id': 'ROOT.FLOW360', 'parentFolderId': None, 'createdAt': '2025-06-02T14:34:12.317054Z', 'updatedAt': '2025-06-02T14:34:12.317054Z'}], nodesIn